In [ ]:
class model_encoder_legecy:
    def __init__(self, df_main, n_components=35, shares={
                                        'sm_name':{'de_x':35},
                                        'cell_type': {'de_x':6}
                                        },
                                        te_params = dict(min_samples_leaf=20, smoothing=100),
                                        grn_model=None
                                        ):  
        # target df to encode
        self.df_main = df_main
        self.grn_model = grn_model
        # regressor
        self.emb_model = RandomForestRegressor(n_estimators=100, random_state=random_state)
        # reducer
        self.reducer = TruncatedSVD(n_components=n_components, n_iter=12, random_state=random_state)
        self.Y = self.reducer.fit_transform(df_main)
        # encoder 
        self.enc = ce.TargetEncoder(**te_params) #[0, 0.05, 0.5, 0.1,0.9,10]
        # encode data
        var_x_name = list(shares['sm_name'].keys())[0]
        n_components = shares['sm_name'][var_x_name]
        y_sm_name = self.create_feature_space(self.df_main, var_x=var_x_name, n=n_components)
        
        # cell type
        var_x_name = list(shares['cell_type'].keys())[0]
        n_components = shares['cell_type'][var_x_name]
        y_cell_type = self.create_feature_space(self.df_main, var_x=var_x_name, n=n_components)
        # the order is important
        self.emb_data = {'sm_name':y_sm_name, 'cell_type': y_cell_type}
        self.determine_X()
        print(self.X.shape, self.X_submit.shape)

    def determine_X(self):
        for i, (name, y) in enumerate(self.emb_data.items()):
            for i_target in tqdm.tqdm(range(y.shape[1])):
                if i_target == 0:
                    X_encoded = self.enc.fit_transform(self.df_main.reset_index()[name], y[:,i_target])
                    X_submit_encoded = self.enc.transform(id_map.reset_index()[name])
                else:
                    X_encoded_tmp = self.enc.fit_transform(self.df_main.reset_index()[name], y[:,i_target])
                    X_encoded = np.concatenate( [X_encoded, X_encoded_tmp], axis = 1)
                    X_encoded_tmp = self.enc.transform(id_map.reset_index()[name])
                    X_submit_encoded = np.concatenate([X_submit_encoded, X_encoded_tmp], axis = 1)
            if i == 0:
                X = X_encoded
                X_submit = X_submit_encoded
            else:
                X = np.concatenate([X, X_encoded], axis = 1)
                X_submit = np.concatenate([X_submit, X_submit_encoded], axis = 1)

        self.X = X
        self.X_submit = X_submit
    def create_feature_space(self, df_to_encode, var_x='sm_name', n=35):
        
        if var_x == 'de_x':
            return TruncatedSVD(n_components=n, n_iter=12, random_state=random_state).fit_transform(df_to_encode.values)

        elif var_x == 'tf_x':
            df_main_c = df_to_encode.copy()
            net = self.grn_model
            tf_act = enrich_tfs(df_main_c, net)

            tf_act = TruncatedSVD(n_components=n, n_iter=12, random_state=random_state).fit_transform(tf_act)
            return tf_act
        raise ValueError('define feature space')
    def validate(self, mask_tr, mask_va):
        # print(mask_tr)
        X_tr, Y_tr = self.X[mask_tr,:], self.Y[mask_tr,:]
        X_va = self.X[mask_va,:]
        self.emb_model.fit(X_tr, Y_tr)
        return self.reducer.inverse_transform(self.emb_model.predict(X_va))
    def calculate_y_submit(self):
        self.emb_model.fit(self.X, self.Y)
        return self.reducer.inverse_transform(self.emb_model.predict(self.X_submit))

class model_encoder:
    def __init__(self, df_main, n_components_y=35, 
                 n_highlyvarible_tfs = 100,
                    te_params = dict(min_samples_leaf=20, smoothing=100),
                    grn_model=None
                    ):  
        # target df to encode
        self.n_highlyvarible_tfs = n_highlyvarible_tfs
        self.df_main = df_main
        self.grn_model = grn_model
        # regressor
        self.emb_model = RandomForestRegressor(n_estimators=100, random_state=random_state)
        # reducer
        self.reducer = TruncatedSVD(n_components=n_components_y, n_iter=12, random_state=random_state)
        self.Y = self.reducer.fit_transform(df_main)
        # encoder 
        self.enc = ce.TargetEncoder(**te_params) #[0, 0.05, 0.5, 0.1,0.9,10]
        # encode data
        self.emb_data = self.create_feature_space(self.df_main)
        self.determine_X()
        print(self.X.shape)
    def determine_X(self):
        features = self.df_main.index.names
        feature_x = self.emb_data
        hot_encoding_features = [feature for feature in features if (feature != 'sm_name')]
        encoder = OneHotEncoder()
        for i, feature in tqdm.tqdm(enumerate(hot_encoding_features)):
            X_encoded = encoder.fit_transform(self.df_main.reset_index()[feature].values.reshape(-1,1)).toarray()
            X_submit_encoded = encoder.transform(id_map[feature].values.reshape(-1,1)).toarray()
            # print(X_encoded)
            # aa
            if i == 0:
                X = X_encoded
                X_submit = X_submit_encoded
            else:
                X = np.concatenate([X, X_encoded], axis = 1)
                X_submit = np.concatenate([X_submit, X_submit_encoded], axis = 1)
        n_components_sm_name = 100
        n_components_sm_name = min([n_components_sm_name, feature_x.shape[1]])
        for i_target in tqdm.tqdm(range(n_components_sm_name)):
            if i_target == 0:
                # print(feature_x[:,i_target])
                X_encoded = self.enc.fit_transform(self.df_main.reset_index()['sm_name'], feature_x[:,i_target])
                X_submit_encoded = self.enc.transform(id_map.reset_index()['sm_name'])
            else:
                X_encoded_tmp = self.enc.fit_transform(self.df_main.reset_index()['sm_name'], feature_x[:,i_target])
                X_encoded = np.concatenate( [X_encoded, X_encoded_tmp], axis = 1)
                X_encoded_tmp = self.enc.transform(id_map.reset_index()['sm_name'])
                X_submit_encoded = np.concatenate([X_submit_encoded, X_encoded_tmp], axis = 1)
        X = np.concatenate([X, X_encoded], axis = 1)
        X_submit = np.concatenate([X_submit, X_submit_encoded], axis = 1)
        self.X = X
        self.X_submit = X_submit
    def calculate_y_submit(self):
        self.emb_model.fit(self.X, self.Y)
        return self.reducer.inverse_transform(self.emb_model.predict(self.X_submit))
    def create_feature_space(self, df_to_encode):
        if self.grn_model is None:
            n = 50
            return TruncatedSVD(n_components=n, n_iter=12, random_state=random_state).fit_transform(df_to_encode.values)
        else:
            n = self.n_highlyvarible_tfs
            df_main_c = df_to_encode.copy()
            net = self.grn_model
            tf_act = enrich_tfs(df_main_c, net)
            # tf_act = TruncatedSVD(n_components=n, n_iter=12, random_state=random_state).fit_transform(tf_act)
            variances = tf_act.var()
            sorted_variances = variances.sort_values(ascending=False)
            top_tfs = sorted_variances.head(n).index.tolist()
            tf_act = tf_act[top_tfs].values
            return tf_act
        raise ValueError('define feature space')
    def validate(self, mask_tr, mask_va):
        X_tr, Y_tr = self.X[mask_tr,:], self.Y[mask_tr,:]
        X_va = self.X[mask_va,:]
        self.emb_model.fit(X_tr, Y_tr)
        return self.reducer.inverse_transform(self.emb_model.predict(X_va))
